In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
import re
from datasketch import MinHash,MinHashLSH

In [ ]:
df =pd.read_csv("tweets_2022_abril_junio.csv")
df

In [ ]:
subconjunto = df.copy()
s = 0.4
k = 4

## Limpieza

In [ ]:
spanish_stopwords = stopwords.words('spanish')

In [ ]:
frase = "RT @RudyPetrikowski: ⚠️ Ayúdenme por favor a conseguir la información de las normas o artículos que la CC ha rechazado.\n\nEsto es muy import…"
def eliminar_stopwords(frase):
    palabras = frase.split(" ")
    retorno = []
    for palabra in palabras:
        if not palabra in spanish_stopwords:
            retorno.append(palabra)
    return " ".join(retorno)
def procesar_frase(frase):
    if (frase.startswith("RT")):
        frase = ' '.join(frase.split(": ")[1:])
    
    # sin hashatgs
    frase = re.sub(r"#\w+", "",frase)
    # sin url
    frase = re.sub(r"https\S+", "",frase)
    # sin emojis
    patron_emoji = re.compile("[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U00002600-\U000027BF]|[\U0001F900-\U0001F9FF]|[\U0001F1E0-\U0001F1FF]|[\U0001F1F2-\U0001F1F4]|[\U0001F1E6-\U0001F1FF]")
    frase = re.sub(patron_emoji, '', frase)
    # sin espacios
    frase = re.sub(r"\s+", " ",frase)
    # eliminar tildes
    frase = frase.replace("á","a")
    frase = frase.replace("é","e")
    frase = frase.replace("í","i")
    frase = frase.replace("ó","o")
    frase = frase.replace("ú","u")

    frase = eliminar_stopwords(frase)
    
    # minusculas
    frase = frase.lower()
    frase = frase.strip()
    return frase

procesar_frase(frase)

'️ ayudenme favor conseguir informacion normas articulos cc rechazado. esto import…'

In [ ]:
subconjunto = subconjunto.drop_duplicates(["screen_name","text"]).reset_index(drop=True)
subconjunto = subconjunto[["id","screen_name","text"]]
subconjunto

,id,screen_name,text
0,1512186166438637582,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,1512186202367045642,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,1512186287284924418,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,1512186335754301446,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,1512186407841767424,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...
...,...,...,...
4549967,1526652300709679104,Alebarrera74,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...
4549968,1526641118460334080,gigita29bq,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...
4549969,1526738292011462657,Elizabe81480339,RT @Gonz1Gorjeperez: @tere_marinovic https://t...
4549970,1526855280151056386,CastilloNafla,RT @Gonz1Gorjeperez: @tere_marinovic https://t...


In [ ]:
subconjunto["text"] = subconjunto["text"].apply(procesar_frase)

In [ ]:
def shingle(frase , tamaño_ventana):
    lista = []
    for i in range(len(frase) - tamaño_ventana +1):
        shingle = ' '.join(frase[i:i+tamaño_ventana])
        lista.append(shingle)
    return ','.join(lista)
subconjunto["text"] = subconjunto["text"].str.split().apply(lambda x: np.array(x)).apply(lambda x: shingle(x,k))

In [ ]:
subconjunto = subconjunto[subconjunto['text'].apply(lambda x: len(x.split()) > 0)]

In [ ]:
subconjunto = subconjunto.reset_index(drop=True)

### Tweeets-Usuario

In [ ]:
agrupado = subconjunto.copy()
agrupado = agrupado.reset_index()
agrupado = agrupado.groupby('screen_name')['index'].agg(list)
agrupado = agrupado.rename('indices_en_lista')
agrupado = agrupado.reset_index().rename(columns={'index': 'indices_en_lista'})
agrupado

In [ ]:
def m_caracteristica(subconjun):
    tweets = subconjun
    shingles = tweets['text'].apply(lambda x: np.array(x.split(',')))
    mlb = MultiLabelBinarizer(sparse_output=True)
    return mlb.fit_transform(shingles)

In [ ]:
matrix = m_caracteristica(subconjunto)
matrix


<4019415x5806559 sparse matrix of type '<class 'numpy.int32'>'
	with 35557046 stored elements in Compressed Sparse Row format>

In [ ]:
n_pm= 10

def minhash_vec(sparce_vector):
    minhash = MinHash(n_pm)
    for i in sparce_vector.indices:
        minhash.update(str(i).encode("utf8"))
    return minhash

In [ ]:
thrld = 0.4
lsh = MinHashLSH(thrld,n_pm)
list_hash = []
for i in range(matrix.shape[0]):
    list_hash.append((i,minhash_vec(matrix[i])))

with lsh.insertion_session() as session:
    for i , minhash in list_hash:
        session.insert(i, minhash)

In [ ]:
len(list_hash)

4019415

### Asociamos los tweets a sus buckets similares

In [ ]:
def transformar_lista(lisa_id_tweet):
    similares = set()
    for id_tweet in lisa_id_tweet:
        m = minhash_vec(matrix[id_tweet])
        aprox = lsh.query(m)
        similares.update(aprox)
    return list(similares)

In [ ]:
agrupado['similaritys'] = agrupado['indices_en_lista'].apply(transformar_lista)

### Asociammos los buckets de tweets, a los usuarios

In [ ]:
def m_caracteristica(subconjun):
    sims = subconjun
    shingles = sims['similaritys'].apply(lambda x: np.array(x.split(',')))
    mlb = MultiLabelBinarizer(sparse_output=True)
    return mlb.fit_transform(shingles)

In [ ]:
matrix = m_caracteristica(agrupado)

In [ ]:
n_pm= 50
thrld = 0.4
lsh = MinHashLSH(thrld,n_pm)
list_hash = []
for i in range(matrix.shape[0]):
    list_hash.append((i,minhash_vec(matrix[i])))

with lsh.insertion_session() as session:
    for i , minhash in list_hash:
        session.insert(i, minhash)

In [ ]:
id_usuario = 0
m = minhash_vec(matrix[id_usuario])
usuarios_similares = lsh.query(m)